In [1]:
import torch
import numpy as np
num_inputs = 2
num_examples = 1000
true_w = [2,-3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0,0.01,size=(num_examples,num_inputs)),dtype=torch.float32)
labels = features[:,0]*true_w[0] + features[:,1]*true_w[1] + true_b
labels += torch.tensor(np.random.normal(0,0.01,size=labels.size()),dtype=torch.float32)


In [2]:
labels

tensor([4.2349, 4.1588, 4.1795, 4.2639, 4.1839, 4.2262, 4.2065, 4.2192, 4.2074,
        4.1591, 4.2467, 4.2019, 4.1956, 4.1946, 4.1533, 4.2199, 4.1563, 4.1717,
        4.2598, 4.1546, 4.1212, 4.1980, 4.1880, 4.1860, 4.2178, 4.1840, 4.2227,
        4.2310, 4.2240, 4.2004, 4.2105, 4.2372, 4.1800, 4.2137, 4.2191, 4.2306,
        4.1305, 4.2046, 4.1684, 4.2085, 4.2895, 4.2990, 4.2116, 4.1833, 4.2054,
        4.2393, 4.2444, 4.1347, 4.2098, 4.1732, 4.2267, 4.1666, 4.1521, 4.2245,
        4.1344, 4.2536, 4.1898, 4.2332, 4.1853, 4.1970, 4.2328, 4.1814, 4.1754,
        4.1642, 4.2089, 4.2024, 4.2221, 4.2080, 4.1780, 4.2312, 4.1997, 4.1981,
        4.1322, 4.1882, 4.1479, 4.2254, 4.1854, 4.2089, 4.2156, 4.1408, 4.2340,
        4.1974, 4.1943, 4.2019, 4.1680, 4.1687, 4.2337, 4.1857, 4.1786, 4.2229,
        4.1378, 4.1463, 4.2401, 4.1860, 4.1351, 4.1358, 4.1371, 4.2518, 4.1630,
        4.2191, 4.2202, 4.1702, 4.1731, 4.1891, 4.2104, 4.1836, 4.1689, 4.2203,
        4.2666, 4.2010, 4.2093, 4.2320, 

In [3]:
features

tensor([[ 0.0080,  0.0002],
        [-0.0094,  0.0066],
        [-0.0122,  0.0008],
        ...,
        [ 0.0038,  0.0115],
        [-0.0037, -0.0051],
        [ 0.0116, -0.0086]])

In [65]:
features.shape


torch.Size([1000, 2])

In [66]:
#读取数据
from torch.utils.data import DataLoader,Dataset

dataset =torch.utils.data.TensorDataset(features,labels)

In [67]:
batch_size = 10
dataloader = DataLoader(dataset,batch_size,shuffle=True)

In [68]:
dataloader

In [69]:
for X, y in dataloader:
    print(X, y)
    break

tensor([[ 0.0006, -0.0050],
        [-0.0051,  0.0009],
        [-0.0015, -0.0055],
        [-0.0083, -0.0072],
        [ 0.0045,  0.0154],
        [-0.0139, -0.0034],
        [ 0.0010, -0.0043],
        [ 0.0140, -0.0030],
        [ 0.0013, -0.0081],
        [-0.0048,  0.0087]]) tensor([4.2179, 4.1769, 4.2047, 4.2125, 4.1702, 4.1969, 4.2100, 4.2310, 4.2348,
        4.1521])


In [76]:
#定义模型
import torch.nn as nn
class liner(nn.Module):
    def __init__(self,n_features):
        super(liner,self).__init__()
        self.fc = nn.Linear(n_features,1)

    def forward(self,x):
        y = self.fc(x)
        return y
model = liner(num_inputs)
print(model)
for param in model.parameters():
    print(param)

liner(
  (fc): Linear(in_features=2, out_features=1, bias=True)
)
Parameter containing:
tensor([[-0.6447, -0.3243]], requires_grad=True)
Parameter containing:
tensor([-0.0933], requires_grad=True)


In [77]:
#初始化模型参数
from torch.nn import init
for name,param in model.named_parameters():
    if 'weight' in name:
        init.normal_(param,mean=0,std=0.01)
        print(name,param)
    if 'bias' in name:
        init.constant_(param, val=0)
        print(name,param)

fc.weight Parameter containing:
tensor([[-0.0013, -0.0016]], requires_grad=True)
fc.bias Parameter containing:
tensor([0.], requires_grad=True)


In [78]:
import torch.optim as optim
loss = nn.MSELoss()
optim = optim.SGD(model.parameters(),lr=0.03)

In [79]:
num_epochs = 3
for epoch in range(1,num_epochs+1):
    for X,y in dataloader:
        out = model(X)
        l = loss(out,y.view(-1,1))
        optim.zero_grad()
        l.backward()
        optim.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.001170
epoch 2, loss: 0.001387
epoch 3, loss: 0.001057


In [80]:
model.parameters()

<generator object Module.parameters at 0x000001A29E454510>

In [81]:
print(model.named_parameters())

<generator object Module.named_parameters at 0x000001A29E4549E0>


In [82]:
for name,param in model.named_parameters():
    print(name,param)

fc.weight Parameter containing:
tensor([[ 0.0029, -0.0039]], requires_grad=True)
fc.bias Parameter containing:
tensor([4.2026], requires_grad=True)
